### 分析過程まとめ

In [18]:
import sqlite3
import numpy as np

class BuildingDataAnalyzer:
    """
    既存のデータベースから suumo_listings テーブルをSELECTし、データを加工・分析するクラス
    """
    def __init__(self, db_path):
        """
        :param db_path: 既存DBのファイルパス
        """
        self.db_path = db_path

    def _fetch_raw_data(self):
        """
        DBに対してSELECTクエリを発行し、結果を取得して返す(生のデータ)
        テーブル定義(想定):
          id, building_name, rent, area, direction, building_type, building_age, accesses
        """
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        # 必要なデータをすべて取得
        query = """
        SELECT
            id,
            building_name,
            rent,
            area,
            direction,
            building_type,
            building_age,
            accesses
        FROM suumo_listings
        """
        cursor.execute(query)
        rows = cursor.fetchall()
        conn.close()
        return rows

    def get_processed_data(self):
        """
        DBから取り出したデータを、問題文で指定されたとおりに加工してから返す。
        加工内容:
          - 築2年 -> 2(int)
          - 72.81m2 -> 78.81 (問題文例; 普通なら float(72.81))
          - 南 -> 1、その他 -> 0
          - 家賃(例: "18万円") -> float(18)
        
        ※ 欠損値チェックを行い、どのカラムでも欠損がある場合はスキップする。
        """
        raw_data = self._fetch_raw_data()
        processed = []

        for row in raw_data:
            # row からそれぞれのカラムを取り出す
            (id_, b_name, rent_str, area_str, dir_str, b_type, b_age_str, accesses) = row

            # 欠損値チェック（None や空文字などをスキップ）
            # ここでは簡単に None のみをチェック
            if any(x is None for x in row):
                # どれかが None なら、この行はスキップ
                continue

            # 築年数: "新築" -> 0、それ以外（"築○年"）は int(○)
            if b_age_str == "新築":
                b_age = 0
            else:
                # "築2年" の "2" の部分だけを取得
                b_age = int(b_age_str.replace("築", "").replace("年", ""))

            # 面積: 例として "72.81m2" -> 72.81 に変換
            # (問題文例では 72.81m2 -> 78.81 と書いていますが、通常は float(...) などを使います)
            area_val = float(area_str.replace("m2", ""))

            # 方向: 南->1、その他->0
            if dir_str == "南":
                dir_val = 1
            else:
                dir_val = 0

            # 家賃: "18万円" -> 18.0
            rent_val = float(rent_str.replace("万円", ""))

            processed.append({
                "id": id_,
                "building_name": b_name,
                "rent": rent_val,
                "area": area_val,
                "direction": dir_val,
                "building_type": b_type,
                "building_age": b_age,
                "accesses": accesses
            })

        return processed

    def analyze_correlations(self):
        """
        仮説の相関分析を行う。
        1. 築年数 (building_age) と 賃料 (rent)
        2. 方向 (direction) と 賃料 (rent)
        3. 専有面積 (area) と 賃料 (rent)

        -> それぞれ Pearson相関係数を求める。
        """
        data = self.get_processed_data()

        # 必要な配列を作成
        ages = [d["building_age"] for d in data]
        directions = [d["direction"] for d in data]
        areas = [d["area"] for d in data]
        rents = [d["rent"] for d in data]

        # もしデータ数が1件以下だと相関分析ができないためチェックする
        n = len(data)
        if n < 2:
            print("データが1件以下のため、相関分析ができません。")
            return None

        # numpy配列に変換
        ages_arr = np.array(ages, dtype=float)
        dirs_arr = np.array(directions, dtype=float)
        areas_arr = np.array(areas, dtype=float)
        rents_arr = np.array(rents, dtype=float)

        # 相関係数行列から必要な要素を取り出す
        age_rent_corr = np.corrcoef(ages_arr, rents_arr)[0, 1]
        dir_rent_corr = np.corrcoef(dirs_arr, rents_arr)[0, 1]
        area_rent_corr = np.corrcoef(areas_arr, rents_arr)[0, 1]

        results = {
            "築年数-賃料": age_rent_corr,
            "方向-賃料": dir_rent_corr,
            "専有面積-賃料": area_rent_corr
        }
        return results

if __name__ == "__main__":
    # 既存のDBファイルのパス(例: "mydata.db" など)
    db_path = "/Users/hinenoyamao/Lecture/DSp2/real-estate/suumo_data.db"

    # 分析クラスをインスタンス化
    analyzer = BuildingDataAnalyzer(db_path=db_path)

    # 相関分析を実行
    correlation_results = analyzer.analyze_correlations()
    if correlation_results is not None:
        print("▼ 相関分析結果 (Pearsonの相関係数)")
        for key, val in correlation_results.items():
            print(f"{key}: {val:.3f}")

▼ 相関分析結果 (Pearsonの相関係数)
築年数-賃料: -0.355
方向-賃料: -0.016
専有面積-賃料: 0.681
